In [1]:
import os
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import plotly
import plotly.graph_objs as go
import cufflinks as cf
cf.set_config_file(offline=True)

In [2]:
base_path = os.path.abspath('../../..')
data_path = os.path.join(base_path, 'data')
database_path = os.path.join(data_path, 'database')
strategy_path = os.path.join(base_path, 'strategy')
check_path = os.path.join(strategy_path, 'check')
r_signal_path = os.path.join(check_path, 'signal')
sys.path.append(strategy_path)

In [3]:
from strategy import CommodityStrategy, EquityStrategy, IRStrategy, EmergingStrategy
from tester import Tester

In [4]:
# EM equity data
emindex = pd.read_csv(os.path.join(data_path, 'totindex-em.csv'), header=0, index_col=0, parse_dates=True)
emindex1 = pd.read_csv(os.path.join(data_path, 'priceindex-em.csv'), header=0, index_col=0, parse_dates=True)
emfut = pd.read_csv(os.path.join(data_path, 'fut1return-em.CSV'), header=0, index_col=0, parse_dates=True)

total_em_ret = emindex.pct_change()
prc_em_ret = emindex1.pct_change()
fut_em_ret = emfut.pct_change()
fut_em_ret = fut_em_ret.reindex(pd.date_range(fut_em_ret.index[0], fut_em_ret.index[-1], freq ='B')) # bizday로 변환

prc_em_ret.loc[total_em_ret.index] = total_em_ret
raw_index = pd.concat([emindex1.loc[:'2012-12-31'], emfut.loc['2013-01-01':]]).iloc[1:]
EMRet = pd.concat([prc_em_ret.loc[:'2012-12-31'], fut_em_ret.loc['2013-01-01':]]).iloc[1:]

EMRet.fillna(0, inplace = True)
EMindex = EMRet
EMindex.iloc[0] = 0
EMindex = (1+ EMindex).cumprod()

raw_index.index.name = 'tdate'
raw_index.columns.name = 'ticker'

EMindex.index.name = 'tdate'
EMRet.index.name = 'tdate'

EMindex.columns.name = 'ticker'
EMRet.columns.name = 'ticker'

raw_index = pd.read_csv(os.path.join(data_path,'priceindex-mon-em.csv') ,header=0, index_col=0, parse_dates=True)
raw_index.index.name = 'tdate'
raw_index.columns.name = 'ticker'

FileNotFoundError: [Errno 2] File b'C:\\Users\\jungyoon.choi\\Desktop\\jungyoon\\project\\risk_premia\\python\\data\\totindex-em.csv' does not exist: b'C:\\Users\\jungyoon.choi\\Desktop\\jungyoon\\project\\risk_premia\\python\\data\\totindex-em.csv'

In [5]:
strategy_name = 'EMDY'

# 필요 데이터 생성
DPS = pd.read_csv(os.path.join(data_path,'DPS-em.csv') ,header=0, index_col=0, parse_dates=True)
DPS1 = pd.read_csv(os.path.join(data_path,'DPS1-em.csv') ,header=0, index_col=0, parse_dates=True)
yield_ = pd.read_csv(os.path.join(data_path,'10Yield-em.csv') ,header=0, index_col=0, parse_dates=True)

In [6]:
cs=0.35,
nopos=0.4
minobs1=12
minobs=60
IDN='out'

In [7]:
group_1 = ['CN', 'KR', 'TW', 'IN']
group_2 = ['MY', 'ID', 'BR', 'MX', 'RU', 'SA']

if IDN == 'out':
    group_2.remove('ID')

DPS = DPS
DPS1 = DPS1
yield_ = yield_
price_index = raw_index.iloc[raw_index.reset_index().groupby(raw_index.index.to_period('M'))[raw_index.index.name].idxmax()]

In [8]:
group_1 = ['CN', 'KR', 'TW', 'IN']
group_2 = ['MY', 'ID', 'BR', 'MX', 'RU', 'SA']

if IDN == 'out':
    group_2.remove('ID')

DPS = DPS
DPS1 = DPS1
yield_ = yield_
price_index = raw_index.iloc[raw_index.reset_index().groupby(raw_index.index.to_period('M'))[raw_index.index.name].idxmax()]

start_date = max(DPS.index[0], price_index.index[0], yield_.index[0])
end_date = min(DPS.index[-1], price_index.index[-1], yield_.index[-1])

DPS = DPS[start_date:end_date]
DPS1 = DPS1[start_date:end_date]
price_index = price_index[start_date:end_date]
yield_ = yield_[start_date:end_date]

DPS[DPS.isnull()] = DPS1[DPS.isnull()]
DYP = price_index.reindex(DPS.index)
DY = DPS / DYP

In [9]:
class EMDY(EmergingStrategy):
    def __init__(self, strategy_name, asset_type):
        super().__init__(strategy_name=strategy_name, asset_type=asset_type)

    def load_strategy_data1(self, table='datastream', origin1='DPS-em', origin2="DPS1-em"):
        self.dps = self._load_strategy_data(table=table, origin=origin1)
        self.dps1 = self._load_strategy_data(table=table, origin=origin2)

    def load_strategy_data2(self, table='bloom', origin='10Yield-em'):
        self.yield10 = self._load_strategy_data(table=table, origin=origin)

    def calculate_signal(self, cs=0.35, nopos=0.4, minobs1=12, minobs=60, IDN='out'):
        group_1 = ['CN', 'KR', 'TW', 'IN']
        group_2 = ['MY', 'ID', 'BR', 'MX', 'RU', 'SA']

        if IDN == 'out':
            group_2.remove('ID')

        DPS = self.dps
        DPS1 = self.dps1
        yield_ = self.yield10
        price_index = self.raw_index.iloc[self.raw_index.reset_index().groupby(self.raw_index.index.to_period('M'))[self.raw_index.index.name].idxmax()]

        start_date = max(DPS.index[0], price_index.index[0], yield_.index[0])
        end_date = min(DPS.index[-1], price_index.index[-1], yield_.index[-1])

        DPS = DPS[start_date:end_date]
        DPS1 = DPS1[start_date:end_date]
        price_index = price_index[start_date:end_date]
        yield_ = yield_[start_date:end_date]

        DPS[DPS.isnull()] = DPS1[DPS.isnull()]

        DYP = price_index.reindex(DPS.index)

        DY = DPS / DYP

        Rvalue = DY.reindex(yield_.index)

        Rvalue = Rvalue.diff(3).dropna(how='all')

        Rvalue2 = DY.reindex(yield_.index) - yield_ / 100

        Rvalue2 = Rvalue2.iloc[3:]

        CSRV1, TSRV1 = self.emdy_csrv_tsrv(Rvalue, Rvalue2, group_1, cs, nopos, minobs, minobs1)
        CSRV2, TSRV2 = self.emdy_csrv_tsrv(Rvalue, Rvalue2, group_2, cs, nopos, minobs, minobs1)

        TSRV = pd.concat([TSRV1, TSRV2], 1).reindex(columns=self.ret.columns).fillna(0)
        CSRV = pd.concat([CSRV1, CSRV2], 1).reindex(columns=self.ret.columns).fillna(0)

        TSRV = TSRV[self.ret.columns]
        CSRV = CSRV[self.ret.columns]

        self.TSRV = TSRV.loc[self.ret.index].fillna(method='ffill').dropna(how='all')
        self.CSRV = CSRV.loc[self.ret.index].fillna(method='ffill').dropna(how='all')

        if self.TSRV.index[0] > self.CSRV.index[0]:
            self.CSRV = self.CSRV.loc[self.TSRV.index[0]:]
        else:
            self.TSRV = self.TSRV.loc[self.CSRV.index[0]:]

    def emdy_csrv_tsrv(self, Rvalue, Rvalue2, use_cols, CS=0.35, nopos=0.4, minobs=60, minobs1=12):
        Rvalue = Rvalue.copy()[use_cols]
        Rvalue2 = Rvalue2.copy()[use_cols]
        RV = Rvalue

        RVrank = RV.iloc[minobs1 - 1:] * 0

        for i in range(minobs - minobs1):
            data = RV.iloc[:(minobs1 + i)].rank(pct=True)
            RVrank.iloc[i] = data.iloc[-1][(len(data) - data.isna().sum()) >= minobs1]

        for i in range(minobs - minobs1, len(RV) - minobs1 + 1):
            data = RV.iloc[i - (minobs - minobs1): (minobs1 + i)].rank(pct=True)
            RVrank.iloc[i] = data.iloc[-1][(len(data) - data.isna().sum()) >= minobs1]

        truecount = np.round((RVrank.shape[-1] - RVrank.isna().sum(1)) * CS)
        truecount = np.repeat(truecount.values, len(RVrank.columns)).reshape(RVrank.shape)

        tiebreaker = RVrank.rolling(5).mean() * 0.0000001
        tiebreaker.iloc[:4] = 0

        tied_RVrank = RVrank + tiebreaker

        CSRV = tied_RVrank.rank(1, method='first')
        CSRV1 = (CSRV.count(1).T + 1 - CSRV.T).T

        CSRVpos = CSRV * 0
        CSRVpos[CSRV <= truecount] = -1
        CSRVpos[CSRV1 <= truecount] = 1
        RV = Rvalue2

        RVrank = RV.iloc[minobs1 - 1:] * 0

        for i in range(len(RV) - minobs1 + 1):
            data = RV.iloc[:(minobs1 + i)].rank(pct=True)
            RVrank.iloc[i] = data.iloc[-1][(len(data) - data.isna().sum()) >= minobs1]

        CSRV = CSRVpos.fillna(0)

        TSRV = RVrank * 0

        TSRV[RVrank > (nopos + (1 - nopos) / 2)] = 1
        TSRV[RVrank < ((1 - nopos) / 2)] = -1
        TSRV.fillna(0, inplace=True)

        return CSRV, TSRV

In [10]:
strategy_name = 'EMDY'
emdy = EMDY(strategy_name = strategy_name, asset_type="EMERGING")
# 필요 데이터 생성
DPS = pd.read_csv(os.path.join(data_path,'DPS-em.csv') ,header=0, index_col=0, parse_dates=True)
DPS1 = pd.read_csv(os.path.join(data_path,'DPS1-em.csv') ,header=0, index_col=0, parse_dates=True)
yield_ = pd.read_csv(os.path.join(data_path,'10Yield-em.csv') ,header=0, index_col=0, parse_dates=True)
#필요 데이터 넣기 
emdy.index = EMindex.copy()
emdy.ret = EMRet.copy()
emdy.dps = DPS
emdy.dps1 = DPS1
emdy.yield10 = yield_
emdy.raw_index = raw_index

emdy.calculate_signal()
CSRV = emdy.CSRV
TSRV = emdy.TSRV
# R signal과 비교
R_TSRV =  pd.read_csv(os.path.join(r_signal_path, f'{strategy_name}_TSRV.csv'),
                      index_col = 'Unnamed: 0')

(R_TSRV- TSRV).abs().sum(1).iplot()

R_CSRV =  pd.read_csv(os.path.join(r_signal_path, f'{strategy_name}_CSRV.csv'),
                      index_col = 'Unnamed: 0')

(R_CSRV- CSRV).abs().sum(1).iplot()

emdy.set_rebalance_period(ts_freq='month', cs_freq='month')
emdy.set_portfolio_parameter(cs_strategy_type='notional')
emdy.make_portfolio()

2019-12-27 13:18:28,784 - EMDY - INFO - [STEP 0] START LOGGING EMDY
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike



2019-12-27 13:18:30,597 - EMDY - INFO - [STEP 2] SET REBALANCE PERIOD
2019-12-27 13:18:30,597 - EMDY - INFO - [STEP 4] SET PORTFOLIO PARAMETER
2019-12-27 13:18:30,598 - EMDY - INFO - [STEP 5] MAKE PORTFOLIO
2019-12-27 13:18:30,598 - EMDY - INFO - [STEP 5 - 1] CALCULATE VOLATILITY
2019-12-27 13:19:01,337 - EMDY - INFO - [STEP 5 - 2] MAKE TS POSITION
2019-12-27 13:19:01,338 - EMDY - INFO - [STEP 5 - 2 - 1] ALIGN TS POSITION WITH REBALANCE DAY
2019-12-27 13:19:02,722 - EMDY - INFO - [STEP 5 - 2 - 2] TARGET VOL CONTROL to TS POSITION
2019-12-27 13:19:02,727 - EMDY - INFO - [STEP 5 - 2 - 3] STRATEGY LEVEL VOL CONTROL to TS POSITION
2019-12-27 13:19:03,269 - EMDY - INFO - [STEP 5 - 2 - 4] MAKE FINAL TS POSITION
2019-12-27 13:19:03,272 - EMDY - INFO - [STEP 5 - 3] MAKE CS POSITION
2019-12-27 13:19:03,272 - EMDY - INFO - [STEP 5 - 3 - 1] NO TARGET VOL CONTROL to CS POSITION
2019-12-27 13:19:03,273 - EMDY - INFO - [STEP 5 - 3 - 2] ALIGN CS POSITION WITH REBALANCE DAY
2019-12-27 13:19:05,342 - E

In [11]:
emdy.std

ticker,CN,KR,TW,IN,MY,ID,BR,MX,RU,SA
tdate,,,,,,,,,,
1995-05-15,0.211533,0.206620,0.205814,0.196448,0.268654,0.206187,0.670288,0.377754,0.382747,0.144596
1995-05-16,0.212183,0.206648,0.205861,0.196635,0.269948,0.206227,0.670668,0.378436,0.384199,0.144756
1995-05-17,0.209428,0.205896,0.205648,0.198254,0.271261,0.204593,0.665966,0.376315,0.374495,0.144249
1995-05-18,0.209763,0.206004,0.201613,0.200010,0.270478,0.200880,0.649869,0.375487,0.379684,0.142784
1995-05-19,0.206650,0.200546,0.201541,0.200417,0.269223,0.192811,0.642047,0.368198,0.378695,0.143622
...,...,...,...,...,...,...,...,...,...,...
2018-05-03,0.251948,0.162306,0.175176,0.126286,0.104854,0.172873,0.274034,0.134586,0.301092,0.172570
2018-05-04,0.252495,0.162498,0.174636,0.126181,0.106037,0.174799,0.273979,0.133669,0.301438,0.172711
2018-05-07,0.254056,0.162498,0.174905,0.126816,0.106490,0.176712,0.275355,0.134641,0.302179,0.173137
